# Lab 4.5.2: Agent Playground with Streamlit

**Module:** 4.5 - Demo Building & Prototyping  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Build multi-page Streamlit applications
- [ ] Implement session state for conversation persistence
- [ ] Visualize agent reasoning and tool calls
- [ ] Use caching effectively for model loading
- [ ] Deploy to Streamlit Cloud

---

## 📚 Prerequisites

- Completed: Module 3.6 (AI Agents)
- Knowledge of: Python, basic Streamlit, agent concepts
- Installed: `streamlit`, `ollama`

---

## 🌍 Real-World Context

AI agents are powerful but **opaque**. When an agent takes 30 seconds to respond, what's happening inside? Which tools did it use? What was it "thinking"?

Companies like [LangSmith](https://smith.langchain.com/) and [Weights & Biases](https://wandb.ai/) offer commercial solutions for agent observability. But for demos and debugging, a simple visualization tool is invaluable.

In this lab, we'll build an "Agent Playground" that:
- Shows tool calls in real-time
- Displays the agent's "thinking" process
- Lets users toggle tools on/off
- Tracks conversation history

---

## 🧒 ELI5: Why Streamlit?

> **Gradio vs Streamlit is like McDonald's vs Chipotle:**
>
> - **Gradio (McDonald's)**: Super fast, standard menu, you know exactly what you're getting. Great for quick demos with inputs→outputs.
>
> - **Streamlit (Chipotle)**: More choices, you build your own thing, takes a bit longer but more customizable. Great for dashboards and multi-page apps.
>
> **When to use which:**
> - Gradio: ML demos, API wrappers, quick prototypes
> - Streamlit: Dashboards, data apps, complex multi-page experiences

---

## Part 1: Streamlit Fundamentals

### Understanding Streamlit's Execution Model

**Critical concept:** Streamlit reruns your **entire script** whenever:
- User interacts with a widget
- Session state changes
- You call `st.rerun()`

This is very different from Gradio's event-driven model!

### 🧒 ELI5: Streamlit's Rerun Model

> Imagine you're drawing on an Etch-a-Sketch. Every time you turn a knob (user input), the screen **clears** and you redraw **everything** from scratch.
>
> That sounds slow, but Streamlit is smart - it caches expensive operations so you don't actually recalculate everything. Think of it as having a photo of your last drawing that you can quickly trace over.

### Running Streamlit

Unlike Gradio which runs in Jupyter, Streamlit apps run from the command line:

```bash
streamlit run app.py
```

We'll write our code in cells and then combine into `.py` files.

In [ ]:
# Install dependencies
# !pip install streamlit>=1.30.0 ollama>=0.1.0

import streamlit as st
print(f"Streamlit version: {st.__version__}")

In [ ]:
# Basic Streamlit concepts - we'll write these to files
# This cell shows the basic patterns

basic_app = '''
import streamlit as st

# Page config must be first Streamlit command
st.set_page_config(
    page_title="My App",
    page_icon="🤖",
    layout="wide"
)

# Title
st.title("Hello Streamlit!")

# Columns for side-by-side layout
col1, col2 = st.columns(2)

with col1:
    st.header("Input")
    name = st.text_input("Your name")
    age = st.slider("Your age", 0, 100, 25)
    
with col2:
    st.header("Output")
    if name:
        st.write(f"Hello, {name}! You are {age} years old.")
    else:
        st.write("Enter your name to see a greeting.")

# Sidebar
with st.sidebar:
    st.header("Settings")
    show_debug = st.checkbox("Show debug info")
    
if show_debug:
    st.write("Debug mode enabled!")
    st.json({"name": name, "age": age})
'''

print("Basic Streamlit app structure:")
print(basic_app)

---

## Part 2: Session State - The Key to Persistence

Since Streamlit reruns everything, how do we keep data between reruns?

**Answer: `st.session_state`** - a dictionary that persists across reruns.

### 🧒 ELI5: Session State

> Imagine you're playing a video game where the console resets every second. How do you keep your score?
>
> You write it on a sticky note (session_state) before the reset, and read it back after!
>
> - `st.session_state.score = 100` → Write to sticky note
> - `score = st.session_state.score` → Read from sticky note

In [ ]:
# Session state example
session_state_app = '''
import streamlit as st

st.title("Counter with Session State")

# Initialize state (only runs if key doesn't exist)
if "count" not in st.session_state:
    st.session_state.count = 0

# Display current count
st.write(f"Count: {st.session_state.count}")

# Buttons modify session state
col1, col2, col3 = st.columns(3)

with col1:
    if st.button("Increment"):
        st.session_state.count += 1
        st.rerun()  # Force rerun to show new value
        
with col2:
    if st.button("Decrement"):
        st.session_state.count -= 1
        st.rerun()
        
with col3:
    if st.button("Reset"):
        st.session_state.count = 0
        st.rerun()

# Show all session state (debugging)
with st.expander("Session State Debug"):
    st.write(dict(st.session_state))
'''

print("Session state patterns:")
print(session_state_app)

### Chat History with Session State

For chat apps, we store the conversation history in session state:

In [ ]:
# Chat with session state
chat_app = '''
import streamlit as st
import ollama

st.title("💬 Simple Chat")

# Initialize message history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Chat input (special Streamlit component)
if prompt := st.chat_input("Say something..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.write(prompt)
    
    # Generate response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = ollama.chat(
                model="llama3.2:3b",
                messages=st.session_state.messages
            )
            reply = response["message"]["content"]
            st.write(reply)
    
    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": reply})

# Clear button in sidebar
with st.sidebar:
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()
'''

print("Chat app with session state:")
print(chat_app)

---

## Part 3: Caching - Speed Up Your App

Loading models on every rerun would be impossibly slow. Streamlit provides two caching decorators:

| Decorator | Use For | Persists Across |
|-----------|---------|----------------|
| `@st.cache_data` | Data, computations | Sessions |
| `@st.cache_resource` | Models, connections | App lifetime |

### 🧒 ELI5: Caching

> **`@st.cache_data`** is like putting leftovers in the fridge. You made dinner (computed something), now you store it to eat later without cooking again.
>
> **`@st.cache_resource`** is like leaving the stove on (safely!). The stove (model) is ready to cook instantly without waiting to heat up.

In [ ]:
# Caching examples
caching_code = '''
import streamlit as st
import ollama
import time

# Cache expensive data computations
@st.cache_data(ttl=3600)  # Cache for 1 hour
def load_data(filepath):
    """Load and process data. Cached based on filepath."""
    # This only runs once per unique filepath
    import pandas as pd
    data = pd.read_csv(filepath)
    return data.describe()

# Cache model/client connections
@st.cache_resource
def get_ollama_client():
    """Initialize Ollama client once."""
    # This only runs once per app lifetime
    return ollama.Client()

@st.cache_resource
def load_embedding_model():
    """Pre-load embedding model."""
    client = get_ollama_client()
    # Warm up the model by running a test embedding
    client.embeddings(model="qwen3-embedding:8b", prompt="test")
    return client

# Use in app
st.title("Caching Demo")

# This is instant after first load
client = get_ollama_client()

# Show cache info
st.write("Model loaded and cached!")
'''

print("Caching patterns:")
print(caching_code)

---

## Part 4: Building the Agent Playground

Now let's build our multi-page Agent Playground!

### Multi-Page App Structure

```
agent_playground/
├── Home.py                 # Main entry point
├── pages/
│   ├── 1_💬_Chat.py       # Agent chat interface
│   ├── 2_🔧_Tools.py      # Tool configuration
│   └── 3_📊_History.py    # Session history & analytics
├── utils/
│   ├── __init__.py
│   └── agent.py           # Agent logic
└── .streamlit/
    └── config.toml        # Streamlit configuration
```

Streamlit automatically creates navigation from files in `pages/`!

In [ ]:
import os

# Create the app structure
app_dir = "agent_playground"
os.makedirs(f"{app_dir}/pages", exist_ok=True)
os.makedirs(f"{app_dir}/utils", exist_ok=True)
os.makedirs(f"{app_dir}/.streamlit", exist_ok=True)

print(f"Created directory structure in {app_dir}/")

In [ ]:
# 1. Create the agent utility module
agent_utils = '''
"""Agent utilities for the playground."""

import json
import math
import datetime
from typing import Dict, List, Any, Optional, Callable
import ollama


# ===== TOOL DEFINITIONS =====

AVAILABLE_TOOLS = {
    "calculator": {
        "name": "calculator",
        "description": "Perform mathematical calculations. Input should be a valid math expression.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The math expression to evaluate, e.g., \'2 + 2\' or \'sin(3.14)\'"}
            },
            "required": ["expression"]
        }
    },
    "datetime": {
        "name": "get_datetime",
        "description": "Get the current date and time.",
        "parameters": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    "weather": {
        "name": "get_weather",
        "description": "Get weather information for a location (mock data).",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city name, e.g., \'San Francisco\'"
                }
            },
            "required": ["location"]
        }
    },
    "web_search": {
        "name": "web_search",
        "description": "Search the web for information (mock data).",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query"
                }
            },
            "required": ["query"]
        }
    }
}


# ===== TOOL IMPLEMENTATIONS =====

def execute_calculator(expression: str) -> str:
    """Safely evaluate a math expression."""
    try:
        # Safe math namespace
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round,
            "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "sqrt": math.sqrt, "log": math.log, "log10": math.log10,
            "pi": math.pi, "e": math.e,
            "pow": pow, "exp": math.exp
        }
        result = eval(expression, safe_dict)
        return f"Result: {result}"
    except Exception as e:
        return f"Error evaluating \'{expression}\': {str(e)}"


def execute_datetime() -> str:
    """Get current date and time."""
    now = datetime.datetime.now()
    return now.strftime("Current date and time: %Y-%m-%d %H:%M:%S")


def execute_weather(location: str) -> str:
    """Mock weather data."""
    # In a real app, call a weather API
    import random
    conditions = ["Sunny", "Cloudy", "Rainy", "Partly Cloudy"]
    temp = random.randint(50, 85)
    return f"Weather in {location}: {random.choice(conditions)}, {temp}°F"


def execute_web_search(query: str) -> str:
    """Mock web search."""
    # In a real app, use a search API
    return f"Search results for \'{query}\': [Mock] Found 10 results about {query}. Top result: Wikipedia article on {query}."


TOOL_EXECUTORS = {
    "calculator": execute_calculator,
    "get_datetime": execute_datetime,
    "get_weather": execute_weather,
    "web_search": execute_web_search
}


# ===== AGENT CLASS =====

class Agent:
    """
    A simple tool-using agent.
    
    This agent can:
    1. Receive user messages
    2. Decide which tools to use
    3. Execute tools and incorporate results
    4. Generate final response
    """
    
    def __init__(self, model: str = "llama3.2:3b", enabled_tools: List[str] = None):
        self.model = model
        self.enabled_tools = enabled_tools or list(AVAILABLE_TOOLS.keys())
        self.conversation_history: List[Dict] = []
        self.tool_calls_log: List[Dict] = []
        self.thinking_log: List[str] = []
    
    def get_active_tools(self) -> List[Dict]:
        """Get tool definitions for enabled tools."""
        return [
            AVAILABLE_TOOLS[tool]
            for tool in self.enabled_tools
            if tool in AVAILABLE_TOOLS
        ]
    
    def execute_tool(self, tool_name: str, args: Dict) -> str:
        """Execute a tool and return the result."""
        executor = TOOL_EXECUTORS.get(tool_name)
        if not executor:
            return f"Unknown tool: {tool_name}"
        
        try:
            if tool_name == "calculator":
                return executor(args.get("expression", ""))
            elif tool_name == "get_datetime":
                return executor()
            elif tool_name == "get_weather":
                return executor(args.get("location", "Unknown"))
            elif tool_name == "web_search":
                return executor(args.get("query", ""))
            else:
                return f"No executor for {tool_name}"
        except Exception as e:
            return f"Tool error: {str(e)}"
    
    def chat(self, user_message: str) -> Dict[str, Any]:
        """
        Process a user message and generate a response.
        
        Returns a dict with:
        - content: The final response text
        - tool_calls: List of tools called
        - thinking: Any reasoning/thinking text
        """
        # Add user message to history
        self.conversation_history.append({
            "role": "user",
            "content": user_message
        })
        
        # Build system prompt with tool info
        tools = self.get_active_tools()
        tool_descriptions = "\n".join([
            f"- {t['name']}: {t['description']}"
            for t in tools
        ])
        
        system_prompt = f"""You are a helpful AI assistant with access to tools.

Available tools:
{tool_descriptions}

To use a tool, respond with:
<tool>tool_name</tool>
<args>{{"param": "value"}}</args>

You can use multiple tools. After getting tool results, provide your final answer.
If you don't need tools, just respond directly.

Think step by step about what the user needs."""
        
        messages = [{"role": "system", "content": system_prompt}]
        messages.extend(self.conversation_history)
        
        # First LLM call - decide on tools
        response = ollama.chat(
            model=self.model,
            messages=messages
        )
        
        assistant_content = response["message"]["content"]
        tool_calls = []
        thinking = ""
        
        # Parse tool calls from response
        import re
        tool_pattern = r"<tool>(.*?)</tool>\s*<args>(.*?)</args>"
        matches = re.findall(tool_pattern, assistant_content, re.DOTALL)
        
        if matches:
            # Extract thinking (text before first tool call)
            first_tool_pos = assistant_content.find("<tool>")
            if first_tool_pos > 0:
                thinking = assistant_content[:first_tool_pos].strip()
            
            # Execute tools
            tool_results = []
            for tool_name, args_str in matches:
                tool_name = tool_name.strip()
                try:
                    args = json.loads(args_str.strip())
                except:
                    args = {}
                
                result = self.execute_tool(tool_name, args)
                tool_call = {
                    "tool": tool_name,
                    "args": args,
                    "result": result
                }
                tool_calls.append(tool_call)
                tool_results.append(f"Tool {tool_name} result: {result}")
            
            # Add tool results and get final response
            messages.append({"role": "assistant", "content": assistant_content})
            messages.append({"role": "user", "content": "Tool results:\n" + "\n".join(tool_results) + "\n\nNow provide your final answer based on these results."})
            
            final_response = ollama.chat(
                model=self.model,
                messages=messages
            )
            final_content = final_response["message"]["content"]
        else:
            final_content = assistant_content
        
        # Log and update history
        self.tool_calls_log.extend(tool_calls)
        if thinking:
            self.thinking_log.append(thinking)
        
        self.conversation_history.append({
            "role": "assistant",
            "content": final_content
        })
        
        return {
            "role": "assistant",
            "content": final_content,
            "tool_calls": tool_calls,
            "thinking": thinking
        }
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        self.tool_calls_log = []
        self.thinking_log = []
    
    def get_stats(self) -> Dict:
        """Get agent statistics."""
        return {
            "messages": len(self.conversation_history),
            "tool_calls": len(self.tool_calls_log),
            "enabled_tools": self.enabled_tools,
            "model": self.model
        }
'''

# Write agent utils
with open(f"{app_dir}/utils/agent.py", "w") as f:
    f.write(agent_utils)

# Create __init__.py
with open(f"{app_dir}/utils/__init__.py", "w") as f:
    f.write("from .agent import Agent, AVAILABLE_TOOLS\n")

print("Created agent utility module")

In [ ]:
# 2. Create Home.py - Main entry point
home_py = '''
"""Agent Playground - Home Page"""

import streamlit as st

# Page configuration
st.set_page_config(
    page_title="Agent Playground",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "tool_calls" not in st.session_state:
    st.session_state.tool_calls = []
if "enabled_tools" not in st.session_state:
    st.session_state.enabled_tools = ["calculator", "datetime", "weather", "web_search"]
if "model" not in st.session_state:
    st.session_state.model = "llama3.2:3b"

# Main content
st.title("🤖 Agent Playground")

st.markdown("""
Welcome to the Agent Playground! This app lets you:

- **Chat** with an AI agent that can use tools
- **Visualize** the agent's reasoning process
- **Configure** which tools are available
- **Analyze** conversation history

---

### Getting Started

1. **Configure Tools** → Go to 🔧 Tools to enable/disable tools
2. **Start Chatting** → Go to 💬 Chat to interact with the agent
3. **Review History** → Go to 📊 History to see analytics

---
""")

# Quick stats
col1, col2, col3 = st.columns(3)

with col1:
    st.metric(
        "Messages",
        len(st.session_state.messages),
        help="Total messages in current conversation"
    )

with col2:
    st.metric(
        "Tool Calls",
        len(st.session_state.tool_calls),
        help="Total tool invocations"
    )

with col3:
    st.metric(
        "Active Tools",
        len(st.session_state.enabled_tools),
        help="Number of enabled tools"
    )

# Sidebar info
with st.sidebar:
    st.markdown("### Current Settings")
    st.info(f"**Model:** {st.session_state.model}")
    st.info(f"**Tools:** {', '.join(st.session_state.enabled_tools) or 'None'}")
    
    st.markdown("---")
    st.markdown("*Built with Streamlit | Module 4.5*")
'''

with open(f"{app_dir}/Home.py", "w") as f:
    f.write(home_py)

print("Created Home.py")

In [ ]:
# 3. Create Chat page
chat_page = '''
"""Agent Playground - Chat Page"""

import streamlit as st
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path(__file__).parent.parent))
from utils.agent import Agent, AVAILABLE_TOOLS

st.set_page_config(page_title="Chat - Agent Playground", page_icon="💬", layout="wide")

st.title("💬 Agent Chat")

# Initialize agent in session state
@st.cache_resource
def get_agent(model: str, tools: tuple):
    """Create agent - cached by model and tools."""
    return Agent(model=model, enabled_tools=list(tools))

# Get current settings
model = st.session_state.get("model", "llama3.2:3b")
enabled_tools = st.session_state.get("enabled_tools", list(AVAILABLE_TOOLS.keys()))

# Create agent (tuple for hashability)
agent = get_agent(model, tuple(enabled_tools))

# Restore history from session state
if st.session_state.messages:
    agent.conversation_history = [
        {"role": m["role"], "content": m["content"]}
        for m in st.session_state.messages
    ]

# Layout: Chat on left, Tool visualization on right
chat_col, tool_col = st.columns([2, 1])

with chat_col:
    st.markdown("### Conversation")
    
    # Chat container
    chat_container = st.container(height=500)
    
    with chat_container:
        # Display messages
        for msg in st.session_state.messages:
            with st.chat_message(msg["role"]):
                st.write(msg["content"])
                
                # Show tool calls if present
                if msg.get("tool_calls"):
                    with st.expander("🔧 Tool Calls", expanded=False):
                        for tc in msg["tool_calls"]:
                            st.code(json.dumps(tc, indent=2), language="json")
                
                # Show thinking if present
                if msg.get("thinking"):
                    with st.expander("💭 Thinking", expanded=False):
                        st.markdown(msg["thinking"])
    
    # Chat input
    if prompt := st.chat_input("Ask the agent something..."):
        # Add user message
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # Get agent response
        with st.spinner("Agent is thinking..."):
            response = agent.chat(prompt)
        
        # Store response with metadata
        st.session_state.messages.append(response)
        
        # Update tool calls log
        if response.get("tool_calls"):
            st.session_state.tool_calls.extend(response["tool_calls"])
        
        # Rerun to show new messages
        st.rerun()
    
    # Clear button
    if st.button("🗑️ Clear Chat"):
        st.session_state.messages = []
        st.session_state.tool_calls = []
        agent.clear_history()
        st.rerun()

with tool_col:
    st.markdown("### 🔧 Tool Activity")
    
    # Show enabled tools
    st.markdown("**Enabled Tools:**")
    for tool in enabled_tools:
        st.markdown(f"- ✅ {tool}")
    
    st.markdown("---")
    
    # Show recent tool calls
    st.markdown("**Recent Tool Calls:**")
    
    if st.session_state.tool_calls:
        for i, tc in enumerate(reversed(st.session_state.tool_calls[-5:])):
            with st.container(border=True):
                st.markdown(f"**{tc['tool']}**")
                st.caption(f"Args: {tc.get('args', {})}")
                st.success(f"Result: {tc.get('result', 'N/A')[:100]}...")
    else:
        st.info("No tool calls yet. Ask the agent something that requires tools!")
    
    # Example prompts
    st.markdown("---")
    st.markdown("**Try these:**")
    examples = [
        "What is 25 * 4 + 100?",
        "What time is it?",
        "What's the weather in Tokyo?",
        "Search for Python tutorials"
    ]
    for ex in examples:
        st.markdown(f"- *{ex}*")
'''

with open(f"{app_dir}/pages/1_💬_Chat.py", "w") as f:
    f.write(chat_page)

print("Created Chat page")

In [ ]:
# 4. Create Tools configuration page
tools_page = '''
"""Agent Playground - Tools Configuration Page"""

import streamlit as st
import sys
from pathlib import Path

sys.path.insert(0, str(Path(__file__).parent.parent))
from utils.agent import AVAILABLE_TOOLS

st.set_page_config(page_title="Tools - Agent Playground", page_icon="🔧", layout="wide")

st.title("🔧 Tool Configuration")

st.markdown("""
Configure which tools the agent can use. Enable tools that match your use case.
""")

# Initialize enabled tools in session state
if "enabled_tools" not in st.session_state:
    st.session_state.enabled_tools = list(AVAILABLE_TOOLS.keys())

# Tool toggle section
st.markdown("### Enable/Disable Tools")

col1, col2 = st.columns(2)

tool_items = list(AVAILABLE_TOOLS.items())
half = len(tool_items) // 2 + len(tool_items) % 2

# Left column
with col1:
    for tool_key, tool_info in tool_items[:half]:
        with st.container(border=True):
            enabled = st.checkbox(
                f"**{tool_info['name']}**",
                value=tool_key in st.session_state.enabled_tools,
                key=f"tool_{tool_key}"
            )
            st.caption(tool_info["description"])
            
            # Update session state
            if enabled and tool_key not in st.session_state.enabled_tools:
                st.session_state.enabled_tools.append(tool_key)
            elif not enabled and tool_key in st.session_state.enabled_tools:
                st.session_state.enabled_tools.remove(tool_key)

# Right column
with col2:
    for tool_key, tool_info in tool_items[half:]:
        with st.container(border=True):
            enabled = st.checkbox(
                f"**{tool_info['name']}**",
                value=tool_key in st.session_state.enabled_tools,
                key=f"tool_{tool_key}"
            )
            st.caption(tool_info["description"])
            
            if enabled and tool_key not in st.session_state.enabled_tools:
                st.session_state.enabled_tools.append(tool_key)
            elif not enabled and tool_key in st.session_state.enabled_tools:
                st.session_state.enabled_tools.remove(tool_key)

st.markdown("---")

# Quick actions
st.markdown("### Quick Actions")

quick_col1, quick_col2, quick_col3 = st.columns(3)

with quick_col1:
    if st.button("Enable All", use_container_width=True):
        st.session_state.enabled_tools = list(AVAILABLE_TOOLS.keys())
        st.rerun()

with quick_col2:
    if st.button("Disable All", use_container_width=True):
        st.session_state.enabled_tools = []
        st.rerun()

with quick_col3:
    if st.button("Reset to Default", use_container_width=True):
        st.session_state.enabled_tools = ["calculator", "datetime"]
        st.rerun()

st.markdown("---")

# Test tools section
st.markdown("### 🧪 Test Tools")
st.markdown("Test individual tools before using them with the agent.")

test_tool = st.selectbox(
    "Select tool to test",
    options=list(AVAILABLE_TOOLS.keys()),
    format_func=lambda x: AVAILABLE_TOOLS[x]["name"]
)

if test_tool == "calculator":
    expr = st.text_input("Expression", value="2 + 2 * 3")
    if st.button("Calculate"):
        from utils.agent import execute_calculator
        result = execute_calculator(expr)
        st.success(result)

elif test_tool == "datetime":
    if st.button("Get Time"):
        from utils.agent import execute_datetime
        result = execute_datetime()
        st.success(result)

elif test_tool == "weather":
    location = st.text_input("Location", value="San Francisco")
    if st.button("Get Weather"):
        from utils.agent import execute_weather
        result = execute_weather(location)
        st.success(result)

elif test_tool == "web_search":
    query = st.text_input("Search Query", value="Python tutorials")
    if st.button("Search"):
        from utils.agent import execute_web_search
        result = execute_web_search(query)
        st.success(result)

# Model settings
st.markdown("---")
st.markdown("### ⚙️ Model Settings")

if "model" not in st.session_state:
    st.session_state.model = "llama3.2:3b"

model = st.selectbox(
    "LLM Model",
    options=["llama3.2:1b", "llama3.2:3b", "qwen3:8b", "mistral:7b"],
    index=["llama3.2:1b", "llama3.2:3b", "qwen3:8b", "mistral:7b"].index(st.session_state.model)
)
st.session_state.model = model

st.info(f"Current model: **{model}**")
'''

with open(f"{app_dir}/pages/2_🔧_Tools.py", "w") as f:
    f.write(tools_page)

print("Created Tools page")

In [ ]:
# 5. Create History/Analytics page
history_page = '''
"""Agent Playground - History & Analytics Page"""

import streamlit as st
import json
from collections import Counter

st.set_page_config(page_title="History - Agent Playground", page_icon="📊", layout="wide")

st.title("📊 Session History & Analytics")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "tool_calls" not in st.session_state:
    st.session_state.tool_calls = []

# Overview metrics
st.markdown("### Overview")

col1, col2, col3, col4 = st.columns(4)

with col1:
    user_msgs = len([m for m in st.session_state.messages if m.get("role") == "user"])
    st.metric("User Messages", user_msgs)

with col2:
    assistant_msgs = len([m for m in st.session_state.messages if m.get("role") == "assistant"])
    st.metric("Agent Responses", assistant_msgs)

with col3:
    st.metric("Tool Calls", len(st.session_state.tool_calls))

with col4:
    if st.session_state.tool_calls:
        tool_counts = Counter(tc["tool"] for tc in st.session_state.tool_calls)
        most_common = tool_counts.most_common(1)[0][0] if tool_counts else "N/A"
    else:
        most_common = "N/A"
    st.metric("Most Used Tool", most_common)

st.markdown("---")

# Tool usage breakdown
st.markdown("### Tool Usage Breakdown")

if st.session_state.tool_calls:
    tool_counts = Counter(tc["tool"] for tc in st.session_state.tool_calls)
    
    # Simple bar chart using Streamlit
    import pandas as pd
    df = pd.DataFrame({
        "Tool": list(tool_counts.keys()),
        "Calls": list(tool_counts.values())
    })
    st.bar_chart(df.set_index("Tool"))
else:
    st.info("No tool calls recorded yet. Start chatting with the agent!")

st.markdown("---")

# Conversation history
st.markdown("### Conversation History")

if st.session_state.messages:
    for i, msg in enumerate(st.session_state.messages):
        with st.expander(
            f"{i+1}. {'👤 User' if msg.get('role') == 'user' else '🤖 Agent'}: {msg.get('content', '')[:50]}...",
            expanded=False
        ):
            st.markdown(f"**Role:** {msg.get('role', 'unknown')}")
            st.markdown(f"**Content:** {msg.get('content', '')}")
            
            if msg.get("tool_calls"):
                st.markdown("**Tool Calls:**")
                st.json(msg["tool_calls"])
            
            if msg.get("thinking"):
                st.markdown("**Thinking:**")
                st.markdown(msg["thinking"])
else:
    st.info("No conversation history yet.")

st.markdown("---")

# Export options
st.markdown("### Export")

export_col1, export_col2 = st.columns(2)

with export_col1:
    if st.session_state.messages:
        json_export = json.dumps({
            "messages": st.session_state.messages,
            "tool_calls": st.session_state.tool_calls
        }, indent=2)
        
        st.download_button(
            "📥 Download as JSON",
            data=json_export,
            file_name="agent_conversation.json",
            mime="application/json"
        )

with export_col2:
    if st.session_state.messages:
        # Markdown export
        md_lines = ["# Agent Conversation\n"]
        for msg in st.session_state.messages:
            role = "User" if msg.get("role") == "user" else "Agent"
            md_lines.append(f"## {role}\n")
            md_lines.append(f"{msg.get('content', '')}\n")
            if msg.get("tool_calls"):
                md_lines.append("\n**Tool Calls:**\n")
                md_lines.append(f"```json\n{json.dumps(msg['tool_calls'], indent=2)}\n```\n")
        
        st.download_button(
            "📄 Download as Markdown",
            data="\n".join(md_lines),
            file_name="agent_conversation.md",
            mime="text/markdown"
        )

st.markdown("---")

# Clear all data
st.markdown("### Danger Zone")

if st.button("🗑️ Clear All History", type="primary"):
    st.session_state.messages = []
    st.session_state.tool_calls = []
    st.success("All history cleared!")
    st.rerun()
'''

with open(f"{app_dir}/pages/3_📊_History.py", "w") as f:
    f.write(history_page)

print("Created History page")

In [ ]:
# 6. Create Streamlit config
config_toml = '''
[theme]
primaryColor = "#007bff"
backgroundColor = "#ffffff"
secondaryBackgroundColor = "#f8f9fa"
textColor = "#262730"
font = "sans serif"

[server]
maxUploadSize = 50
enableCORS = false

[browser]
gatherUsageStats = false
'''

with open(f"{app_dir}/.streamlit/config.toml", "w") as f:
    f.write(config_toml)

print("Created Streamlit config")

In [ ]:
# 7. Create requirements.txt for deployment
requirements = '''streamlit>=1.30.0
ollama>=0.1.0
pandas>=2.0.0
'''

with open(f"{app_dir}/requirements.txt", "w") as f:
    f.write(requirements)

print("\n✅ Agent Playground created!")
print(f"\nTo run: cd {app_dir} && streamlit run Home.py")
print("\nFile structure:")
for root, dirs, files in os.walk(app_dir):
    level = root.replace(app_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f'{subindent}{file}')

---

## Part 5: Running and Testing the App

### Running Locally

```bash
# Navigate to app directory
cd agent_playground

# Run Streamlit
streamlit run Home.py
```

The app will open at `http://localhost:8501`

### Running in Docker

```dockerfile
FROM python:3.11-slim

WORKDIR /app
COPY . .

RUN pip install -r requirements.txt

EXPOSE 8501

CMD ["streamlit", "run", "Home.py", "--server.address", "0.0.0.0"]
```

---

## Part 6: Deploying to Streamlit Cloud

### Step 1: Push to GitHub

```bash
# Initialize git repo
cd agent_playground
git init
git add .
git commit -m "Initial agent playground"

# Create repo on GitHub, then:
git remote add origin https://github.com/YOUR_USERNAME/agent-playground.git
git push -u origin main
```

### Step 2: Connect to Streamlit Cloud

1. Go to [share.streamlit.io](https://share.streamlit.io/)
2. Click "New app"
3. Select your GitHub repo
4. Set main file path: `Home.py`
5. Click "Deploy"

### Step 3: Configure Secrets

If your app needs secrets (API keys, etc.), add them in the Streamlit Cloud dashboard:

```toml
# secrets.toml (in Streamlit Cloud dashboard)
OLLAMA_HOST = "your-ollama-server.com"
```

Access in code:
```python
import streamlit as st
ollama_host = st.secrets["OLLAMA_HOST"]
```

---

## ⚠️ Common Mistakes

### Mistake 1: Not Using Session State

```python
# ❌ Wrong - counter resets on every rerun
counter = 0
if st.button("Increment"):
    counter += 1
st.write(counter)  # Always shows 0 or 1

# ✅ Right - use session state
if "counter" not in st.session_state:
    st.session_state.counter = 0

if st.button("Increment"):
    st.session_state.counter += 1

st.write(st.session_state.counter)  # Persists!
```

---

### Mistake 2: Loading Models Without Caching

```python
# ❌ Wrong - loads on every rerun (slow!)
model = load_heavy_model()

# ✅ Right - cache the resource
@st.cache_resource
def load_model():
    return load_heavy_model()

model = load_model()  # Cached!
```

---

### Mistake 3: Putting st.set_page_config After Other Commands

```python
# ❌ Wrong - page config must be first
import streamlit as st
st.title("My App")
st.set_page_config(page_title="My App")  # Error!

# ✅ Right - page config first
import streamlit as st
st.set_page_config(page_title="My App")
st.title("My App")
```

---

### Mistake 4: Widget Key Collisions

```python
# ❌ Wrong - duplicate keys cause errors
for i in range(3):
    st.text_input("Name")  # Same key for all!

# ✅ Right - unique keys
for i in range(3):
    st.text_input("Name", key=f"name_{i}")
```

---

## 🎉 Checkpoint

You've learned:
- ✅ Streamlit's rerun execution model
- ✅ Session state for persistence
- ✅ Caching strategies for performance
- ✅ Multi-page app structure
- ✅ Agent reasoning visualization
- ✅ Deploying to Streamlit Cloud

---

## 🚀 Challenge (Optional)

Enhance the Agent Playground with:

1. **Streaming responses** - Show tokens as they're generated
2. **Tool latency tracking** - Measure and display how long each tool takes
3. **Conversation branching** - Let users go back and try different responses
4. **Custom tool creation** - UI to define new tools dynamically

<details>
<summary>💡 Hints</summary>

- For streaming: Use `st.write_stream()` with a generator
- For latency: Wrap tool execution in `time.time()` calls
- For branching: Store conversation tree in session state
- For custom tools: Use `st.text_area` for JSON tool definitions
</details>

---

## 📖 Further Reading

- [Streamlit Documentation](https://docs.streamlit.io/)
- [Session State Guide](https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state)
- [Multi-page Apps](https://docs.streamlit.io/develop/concepts/multipage-apps)
- [Streamlit Components](https://streamlit.io/components)
- [Deploy to Streamlit Cloud](https://docs.streamlit.io/deploy/streamlit-community-cloud)

---

## 🧹 Cleanup

In [ ]:
# Files are created on disk - nothing to clean in memory
print("✅ Lab complete!")
print(f"\nApp files are in: {app_dir}/")
print(f"To run: cd {app_dir} && streamlit run Home.py")

---

## ➡️ Next Steps

Continue to [Lab 4.5.3: Portfolio Demo](lab-4.5.3-portfolio-demo.ipynb) to create a polished demo for your capstone project!